In [12]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as LR

with open('.env') as f:
    _data_pth_ = f.readline().strip()
    _data_pth_ = os.path.expanduser(_data_pth_)

### load data

In [13]:
data = pd.read_csv(f'{_data_pth_}/processed/train_transaction.csv', index_col=0)

### Regression

In [14]:
y, X = data['isFraud'], data.drop(columns=['isFraud'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
_lr = LR(penalty='l1', solver='saga') # use LASSO to filter features 
model = _lr.fit(X_train, y_train)

/usr/local/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


### Test

In [15]:
sc_train = model.score(X_train, y_train)
print(sc_train)
sc_test = model.score(X_test, y_test)
print(sc_test)

0.9719785371820826
0.9720441915239713


### dumb classifier test

In [29]:
y, X = data['isFraud'], data.drop(columns=['isFraud'])
y[5:10] = 1
model = _lr.fit(X[:10], y[:10])
sc_dumb = model.score(X_test, y_test)
print(sc_dumb)

0.28671637272358746


/usr/local/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
